In [1]:
import sys
import requests
import json
import datetime
import os
import pandas as pd
import numpy as np
import os
from dateutil.relativedelta import relativedelta
import pytz
import json
import math
import ast
from sys import stdout as out

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
def get_dev_info(device, address):
    
    r = requests.post(address + "/api/auth/login",
                      json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()
    
    # acc_token is the token to be used in the next request
    acc_token = 'Bearer' + ' ' + r['token']
    
    # get devid by serial name
    r1 = requests.get(
        url=address + "/api/tenant/devices?deviceName=" + device,
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    # print(r1)
    label = r1['label']
    devid = r1['id']['id']
    r1 = requests.get(
        url=address + "/api/device/" + devid + "/credentials",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    devtoken = r1['credentialsId']

    
    
    return devid,devtoken,acc_token,label
    



def read_data(acc_token, devid, address, start_time, end_time, descriptors):

        
    r2 = requests.get(
        url=address + "/api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys=" + descriptors + "&startTs=" + start_time + "&endTs=" + end_time + "&agg=AVERAGE,interval=60000&limit=1000000",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    
    if r2:
        df = pd.DataFrame([])
        
        for desc in r2.keys():
            df1 = pd.DataFrame(r2[desc])
            df1.set_index('ts', inplace=True)
            df1.columns = [str(desc)]
            
            df1.reset_index(drop=False, inplace=True)
            df1['ts'] = pd.to_datetime(df1['ts'], unit='ms')
            df1['ts'] = df1['ts'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens')
            df1 = df1.sort_values(by=['ts'])
            df1.reset_index(drop=True, inplace=True)
            df1.set_index('ts', inplace=True, drop=True)            
            
            df = pd.concat([df, df1], axis=1)

        if df.empty:
            df = pd.DataFrame([])
        else:
            for col in df.columns:
                df[col] = df[col].astype('float64')
    else:
        df = pd.DataFrame([])
        # print('Empty json!')
    return df


In [3]:
address = 'https://mi6.meazon.com'
r = requests.post(address + "/api/auth/login",
                      json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()
    
acc_token = 'Bearer' + ' ' + r['token']

local_tz = pytz.timezone('Europe/Athens')
interval = 15
descriptors = 'curA,curB,curC,cnrgA,cnrgB,cnrgC,vltA,vltB,vltC,pwrA,pwrB,pwrC'
end_time = '1718830800000'
excl_devs = ['102.402.002028',
'102.402.002033',
'102.402.002035',
'102.402.002036',
'102.402.002037',
'102.402.002038']

devsFeb = ['102.301.001193',
'102.301.001197',
'102.402.001185',
'102.402.001186',
'102.402.001187',
'102.402.001413',
'102.402.001582',
'102.402.001589',
'102.402.001595',
'102.402.001600',
'102.402.001601',
'102.402.002016',
'102.402.002018',
'102.402.002021']

devsJan = ['102.301.001074',
'102.301.001273',
'102.402.001514',
'102.402.001571',
'102.402.002014',
'102.402.002019',
'102.402.002020',
'102.402.002022',
'102.402.002024',
'102.402.002025',
'102.402.002026',
'102.402.002034',
'102.402.002072']
assetid = '015b8840-6e7c-11ee-bb8e-95f193b12c98' #  Eugenides Foundation : Building asset group from customs hierarchy
r2 = requests.get(url=address + "/api/relations/info?fromId="+assetid+"&fromType=ASSET",headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()

    


In [4]:
excel_file_path = 'evgenidio.xlsx'

with pd.ExcelWriter(excel_file_path) as writer:
    for j in range(0, len(r2)):
        device = r2[j]['toName']
        if device not in excl_devs:
            if device in devsJan:
                start_time = '1704060000000'
            elif device in devsFeb:
                start_time = '1706738400000'
            
            # extract info
            [devid, _, acc_token,label] = get_dev_info(device, address) 
            # read data
            timethres = 7*24*3600000
            svec = np.arange(int(start_time),int(end_time),timethres)

            df = pd.DataFrame([])

            for st in svec:
                en = st+timethres-1
                
                if int(end_time)-en<=0: en = int(end_time)
                tmp = read_data(acc_token, devid, address,  str(st), str(en), descriptors)
                if not tmp.empty:
                    agg_dict = {col: 'mean' for col in tmp.columns}
                    specific_columns = ['cnrgA', 'cnrgB', 'cnrgC']
                    for col in specific_columns:
                        agg_dict[col] = 'max'
                    tmp = tmp.resample(str(interval)+'T').agg(agg_dict)
                    # tmp = tmp.dropna()
                    df = pd.concat([df,tmp])

            del tmp
            df.sort_index(inplace=True)
            for col in ['cnrgA','cnrgB','cnrgC']:
                df[col] = df[col]-df[col].shift()
                df[col].iloc[0] = 0
                df.loc[df[col]<0,col] = np.nan
            for col in ['pwrA','pwrB','pwrC']:
                df.loc[df[col]<0,col] = 0 
            df = df.dropna()
            df = df.rename(columns={'cnrgA':'nrgA','cnrgB':'nrgB','cnrgC':'nrgC'})
            df.index = df.index.tz_localize(None)
            df.to_excel(writer, sheet_name=device)
            
            print(df.head(15))

        


                      curA   curB   curC  nrgA  nrgB  nrgC        vltA  \
ts                                                                       
2024-01-01 00:00:00  0.044  0.046  0.047   0.0   0.0   0.0  239.358667   
2024-01-01 00:15:00  0.044  0.046  0.047   0.0   0.0   0.0  239.617667   
2024-01-01 00:30:00  0.044  0.046  0.047   0.0   0.0   0.0  239.947000   
2024-01-01 00:45:00  0.044  0.046  0.047   0.0   0.0   0.0  240.069333   
2024-01-01 01:00:00  0.044  0.046  0.047   0.0   0.0   0.0  240.500000   
2024-01-01 01:15:00  0.044  0.046  0.047   0.0   0.0   0.0  239.577000   
2024-01-01 01:30:00  0.044  0.046  0.047   0.0   0.0   0.0  239.881000   
2024-01-01 01:45:00  0.044  0.046  0.047   0.0   0.0   0.0  240.451000   
2024-01-01 02:00:00  0.044  0.046  0.047   0.0   0.0   0.0  240.327333   
2024-01-01 02:15:00  0.044  0.046  0.047   0.0   0.0   0.0  240.465000   
2024-01-01 02:30:00  0.044  0.046  0.047   0.0   0.0   0.0  240.879667   
2024-01-01 02:45:00  0.044  0.046  0.0